In [17]:
   def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        all_text = ""
        for page in pdf.pages:
            text = page.extract_text()
            all_text += text + '\n\n'
    return all_text


In [18]:
def process_cite_functions(pdf_path):
#    italics_check=check_italics_in_et_al_citations(pdf_path)
    citation_text= extract_text_from_pdf(pdf_path)
#    matches = extract_pattern(citation_text)
    matches = citations.find_all_citations (citation_text)
    df = pd.DataFrame(matches, columns=['Citations'])
    df.drop_duplicates(subset='Citations', inplace=True)
    
    
    def extract_author_and_year(citation):
        # Define a regular expression pattern to match a year format '2000' or '(2000)'
        year_pattern = r'\b(?:\(\d{4}\)|\d{4})\b'
        year_match = re.search(year_pattern, citation)
        
        if year_match:
            year = year_match.group()
            author = citation.replace(year, '').strip()  # Extract author by removing year
            author = re.sub(r'\(\)', '', author)  # Remove remaining parentheses
        else:
            year = None
            author = citation.strip()

        return {'Author': author, 'Year': year}

    df[['Author', 'Year']] = df['Citations'].apply(extract_author_and_year).apply(pd.Series)
    dfc = df.reset_index(drop=True)
    cita_year = dfc.dropna()
#############In Sentence POSITION OF CITATION ###############################
    def cite_sentence_position(citation):
        pattern =global_pattern
        citations = re.findall(pattern, citation_text)
        #Creating a list to store positions
        positions = []

        for citation in citations:
            #Checking citation position in sentence
            position = None
            if citation_text.startswith(citation):
                position = "At Start Please check!!"
            elif citation_text.endswith(citation):
                position = "✔"
            else:
                position = "✔"
            positions.append((citation, position))
        return {'In Sentence Use': position}
    cita_year[['In Sentence Use']] = cita_year['Citations'].apply(cite_sentence_position).apply(pd.Series)
    specific_pattern = r"([A-Z][a-z]+) et al\.,? \(?(\d{4})\)?"
    generic_pattern = r"[A-Z][a-z]+ (?:et\,?\s?al\.,?|et al\.,?|et\,? al\.,?) \d{4}"

    def check_et_al_syntax(citation):
        if re.search(specific_pattern, citation):
            return '✔'
        elif re.search(generic_pattern, citation):
            return '❌'
        else:
            return 'n/a'

    # Applying the function to create the 'et_al syntax' column
    cita_year['et_al syntax'] = cita_year['Citations'].apply(check_et_al_syntax).apply(pd.Series)

    citedf=HTML(cita_year.to_html(classes='table table-stripped'))
    cita_year.to_csv('citation_data.csv', index=False)
    return { 'Citations in PDF': df}

In [19]:
import citations
import fitz  # PyMuPDF
import pdfplumber
import re
import pandas as pd
from IPython.display import HTML, display


In [20]:
global_pattern = r"\b(?!(?:Although|In|Furthermore|Additionally|Also)\b)(?!(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\b)(?:[A-Z][A-Za-z'`-]+)(?:,? (?:(?:and |& )?(?:[Z-Z][A-Za-z'`-]+)|(?:et al.?)))*(?:,? *(?:19|20)[0-9][0-9](?:, p\.? [0-9]+)?| *\((?:19|20)[0-9][0-9](?:, p\.? [0-9]+)?\))"


In [21]:
 process_cite_functions("C:\\Users\\Glen\\OneDrive - Teesside University\\CIS4006 Advanced Practice\\External Students\\test.pdf")

{'Citations in PDF':                     Citations                 Author  Year
 0              (Dubois, 2018)             (Dubois, )  2018
 1            (Phillips, 2018)           (Phillips, )  2018
 2              (Parker, 2021)             (Parker, )  2021
 3            (Anderson, 2019)           (Anderson, )  2019
 4               (Green, 2018)              (Green, )  2018
 5    (Williams & Evans, 2019)   (Williams & Evans, )  2019
 6   (Garcia & Martinez, 2020)  (Garcia & Martinez, )  2020
 7         (Lee & Smith, 2017)        (Lee & Smith, )  2017
 8       (White & Davis, 2019)      (White & Davis, )  2019
 9      (Morgan & Adams, 2020)     (Morgan & Adams, )  2020
 11    (Johnson & Brown, 2019)    (Johnson & Brown, )  2019
 12    (Nelson & Smith,\n2017)    (Nelson & Smith,\n)  2017
 14  (Robinson & Nelson, 2018)  (Robinson & Nelson, )  2018
 15  (Wright & Martinez, 2020)  (Wright & Martinez, )  2020
 16   (Robinson & Adams, 2019)   (Robinson & Adams, )  2019
 17       (Hall & Ba

In [22]:
 process_cite_functions("C:\\Users\\u0037034\\OneDrive - Teesside University\\CIS4006 Advanced Practice\\External Students\\test.pdf")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\u0037034\\OneDrive - Teesside University\\CIS4006 Advanced Practice\\External Students\\test.pdf'